In [15]:
import snowflake.snowpark.functions as F
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import StructType, StructField, FloatType
from snowflake.snowpark import Session
import os
import json
import pandas as pd
import numpy as np

In [2]:
connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "schema": "SENSOR",
    "database": "THINGSBOARD",
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
}

session = Session.builder.configs(connection_parameters).create()

In [3]:
environmental_df = session.table("ENVIRONMENTAL")

In [4]:
environmental_df.show(5)

-------------------------------------------------------------------------------------------
|"TS"                 |"CO"  |"HUMIDITY"  |"LIGHT"  |"LPG"  |"MOTION"  |"SMOKE"  |"TEMP"  |
-------------------------------------------------------------------------------------------
|2024-01-02 12:30:00  |5.0   |51.1        |False    |7.7    |False     |20.6     |22.7    |
|2024-01-02 12:20:00  |2.7   |75.6        |False    |5.0    |False     |12.9     |19.8    |
|2024-01-02 12:10:00  |5.0   |51.1        |False    |7.7    |False     |20.5     |22.6    |
|2024-01-02 12:00:00  |2.8   |75.8        |False    |5.1    |False     |13.1     |19.8    |
|2024-01-02 11:50:00  |5.0   |51.1        |False    |7.7    |False     |20.5     |22.7    |
-------------------------------------------------------------------------------------------



In [6]:
environmental_df = environmental_df.drop(["HUMIDITY", "LIGHT", "LPG", "MOTION", "SMOKE", "TEMP"])

In [7]:
pandas_df =environmental_df.toPandas()

In [10]:
pandas_df['TS'] = pd.to_datetime(pandas_df['TS'])
pandas_df = pandas_df.sort_values('TS')
pandas_df.set_index('TS', inplace=True)


In [16]:
def create_sequences(data, window_size):
    sequences = []
    targets = []
    for i in range(len(data) - window_size):
        sequences.append(data[i:i+window_size])
        targets.append(data[i+window_size])
    return np.array(sequences), np.array(targets)

window_size = 5 
data = pandas_df['CO'].values
sequences, targets = create_sequences(data, window_size)


In [26]:
import mlflow
import mlflow.tensorflow

mlflow.set_experiment("Time Series Deep Learning")
mlflow.set_tracking_uri("https://sturdy-space-meme-g45wqpx5x49x2vjr-5000.app.github.dev/")


2024/11/22 16:06:47 INFO mlflow.tracking.fluent: Experiment with name 'Time Series Deep Learning' does not exist. Creating a new experiment.


In [28]:
with mlflow.start_run():
    # Log Parameters
    mlflow.log_param("epochs", 50)
    mlflow.log_param("batch_size", 16)
    mlflow.log_param("window_size", window_size)
    

    history = model.fit(
        sequences,
        targets,
        epochs=50,
        batch_size=16,
        validation_split=0.2,
        verbose=1
    )
    
  
    for epoch in range(len(history.history['loss'])):
        mlflow.log_metric("loss", history.history['loss'][epoch], step=epoch)
        mlflow.log_metric("val_loss", history.history['val_loss'][epoch], step=epoch)
    
    # Log the Model
    mlflow.keras.log_model(
        model,
        artifact_path="model",
        registered_model_name="CO_TimeSeries_Model"
    )


Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1140 - val_loss: 0.1060
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1155 - val_loss: 0.1104
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1187 - val_loss: 0.1113
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1191 - val_loss: 0.1113
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1075 - val_loss: 0.1097
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1199 - val_loss: 0.1125
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1154 - val_loss: 0.1078
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1113 - val_loss: 0.1096
Epoch 9/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1018 - val_loss: 0.1074
Epoch 10/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1156 - val_loss: 0.1086
Epoch 11/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1111 - val_loss: 0.1070
Epoch 12/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1172 - val_l

2024/11/22 16:08:47 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/11/22 16:08:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'CO_TimeSeries_Model' already exists. Creating a new version of this model...
2024/11/22 16:08:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CO_TimeSeries_Model, version 2


🏃 View run burly-lark-891 at: https://sturdy-space-meme-g45wqpx5x49x2vjr-5000.app.github.dev/#/experiments/1/runs/446df3dd5c594721a92e6303ac860ae5
🧪 View experiment at: https://sturdy-space-meme-g45wqpx5x49x2vjr-5000.app.github.dev/#/experiments/1


Created version '2' of model 'CO_TimeSeries_Model'.
